In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

In [5]:
titanic_data = pd.read_csv("titanic_data.csv")
titanic_data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,CabinReduced
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",B
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",C
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",C
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",C
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",C


#### funkcja `train_test_split` - służy do losowego podziału danych na zbiór treningowy oraz tetowy.
przyjmowane argumnety: \
\
`X` - dane wejściowe (zmienna niezależna)\
`y` - etykiety (zmienna zależna - przewidywana)\
`test_size` - procent liczby danych przeznaczonych na testy \
`train_size` - procent liczby danych przeznaczonych na zbiór treningowy \
`random_state` - ziarno losowe

In [28]:
col_name = ["cabin", "CabinReduced", "sex"]
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(
    titanic_data[col_name], titanic_data["survived"], test_size=0.2, random_state=0
)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(1047, 3) (262, 3)
(1047,) (262,)


In [29]:
for col in col_name:
    Unique_test = [x for x in X_test[col].unique() if x not in X_train[col].unique()]
    print(len(Unique_test))

24
1
0


Jak możemy zaobserwować, rozkład etykiet dla etykiet z cechy caibn nie jest równomierny w zbiorze treningowym oraz testowym. Może to prowadzić, to błędengo przewidywania wartości zależnej na zbiorze testowym, gydż, model będzie miał do czynienia z etykietami, których nie było w zbiorze trenignowym.

In [36]:
for col in col_name:
    unique_vals = set(X_train[col].dropna().unique()).union(set(X_test[col].dropna().unique()))
    mapping = {val: i for i, val in enumerate(sorted(unique_vals))}
    X_train[col + "_map"] = X_train[col].map(mapping)
    X_test[col + "_map"] = X_test[col].map(mapping)

print(X_train.head())
print(X_test.head())

     cabin CabinReduced     sex  cabin_map  CabinReduced_map  sex_map
1118   NaN          NaN    male        NaN               NaN        1
44     E40            E  female      160.0               4.0        0
1072   NaN          NaN    male        NaN               NaN        1
1130   NaN          NaN  female        NaN               NaN        0
574    NaN          NaN    male        NaN               NaN        1
     cabin CabinReduced     sex  cabin_map  CabinReduced_map  sex_map
1139   NaN          NaN    male        NaN               NaN        1
533    NaN          NaN  female        NaN               NaN        0
459    NaN          NaN    male        NaN               NaN        1
1150   NaN          NaN    male        NaN               NaN        1
393    NaN          NaN    male        NaN               NaN        1


In [46]:
colmap = ['cabin_map', 'CabinReduced_map', 'sex_map']

print("\nLiczba wartości w zbiorze treningowym:")
for col in colmap:
    print(col, len(X_train[col]))
print("Braki w zbiorze treningowym:")
print(X_train[['cabin_map', 'CabinReduced_map', 'sex_map']].isnull().sum())

print("\nLiczba wartości w zbiorze testowym:")
for col in colmap:
    print(col, len(X_test[col]))
print("\nBraki w zbiorze testowym:")
print(X_test[['cabin_map', 'CabinReduced_map', 'sex_map']].isnull().sum())



Liczba wartości w zbiorze treningowym:
cabin_map 1047
CabinReduced_map 1047
sex_map 1047
Braki w zbiorze treningowym:
cabin_map           803
CabinReduced_map    803
sex_map               0
dtype: int64

Liczba wartości w zbiorze testowym:
cabin_map 262
CabinReduced_map 262
sex_map 262

Braki w zbiorze testowym:
cabin_map           211
CabinReduced_map    211
sex_map               0
dtype: int64


W zbiorze treningowym i testowym zauważalna jest duża liczba brakujących wartości w kolumnach cabin_map oraz CabinReduced_map. Szczególnie dotyczy to:

Zbiór treningowy: około 23% braków (1047 ogółem, z czego 803 wartości obecnych).
Zbiór testowy: aż ponad 80% braków w tych kolumnach (262 ogółem, z czego tylko 51 wartości obecnych).
Kolumna sex_map nie zawiera braków w żadnym ze zbiorów, więc nie wymaga uzupełniania.

W kolejnych krokach należy uzupełnić brakujące dane, aby umożliwić modelowi skuteczne uczenie.

In [49]:
for col in colmap:
    X_train[col] = X_train[col].fillna(0).astype(int)
    X_test[col] = X_test[col].fillna(0).astype(int)

print(X_train.head())
print(X_test.head())

     cabin CabinReduced     sex  cabin_map  CabinReduced_map  sex_map
1118   NaN          NaN    male          0                 0        1
44     E40            E  female        160                 4        0
1072   NaN          NaN    male          0                 0        1
1130   NaN          NaN  female          0                 0        0
574    NaN          NaN    male          0                 0        1
     cabin CabinReduced     sex  cabin_map  CabinReduced_map  sex_map
1139   NaN          NaN    male          0                 0        1
533    NaN          NaN  female          0                 0        0
459    NaN          NaN    male          0                 0        1
1150   NaN          NaN    male          0                 0        1
393    NaN          NaN    male          0                 0        1


Zastąpienie brakujących wartości zerami w przypadku kolumn cabin jest poprawną opcją, gdyż jest to zmenna kategoryczna, a nie liczbowa. Dla kolumn takich jak cabin_map, gdzie dane reprezentują pewną kategoryczną lub binarną informację, najlepsze podejścia to:
- zastąpienie wartością spcejlaną np. 0 (-1)
- zastąpienie wartościa najczęściej się powtarzającą

In [53]:
print("Przed mapowaniem")
print("Liczba unikalnych wartości w zbiorze treningowym:")
for col in col_name:
    print(col, len(X_train[col].unique()))
print("Liczba unikalnych wartości w zbiorze testowym:")
for col in col_name:
    print(col, len(X_test[col].unique()))

print("------------------------------")
print("Po mapowaniu")
print("Liczba unikalnych wartości w zbiorze treningowym:")
for col in colmap:
    print(col, len(X_train[col].unique()))
print("Liczba unikalnych wartości w zbiorze testowym:")
for col in colmap:
    print(col, len(X_test[col].unique()))

Przed mapowaniem
Liczba unikalnych wartości w zbiorze treningowym:
cabin 164
CabinReduced 9
sex 2
Liczba unikalnych wartości w zbiorze testowym:
cabin 49
CabinReduced 8
sex 2
------------------------------
Po mapowaniu
Liczba unikalnych wartości w zbiorze treningowym:
cabin_map 163
CabinReduced_map 8
sex_map 2
Liczba unikalnych wartości w zbiorze testowym:
cabin_map 49
CabinReduced_map 7
sex_map 2


Proces redukcji cabin, znacznie zmniejszył różnicę etykiet w zbiorze treningowym w porównaniu do zbioru testowego. Tego właśnie oczekiwaliśmy, aby dążyć do takiego samego rozkładu równomiernego cech w zborze treningowym i zbiorze testowym.
Samo mapowanie, nie wpłyneło w żaden sposób na zmniejszenie różnicy w rozkładach cech, gdyż wartości których nie było w zbiorze treningowym, a w zbiorze testowym były, zostawały zastępowane przez kolejne unikatowe wartości.

Tak, więc cały proces może wpłynąc pozytywnie na końcowy wynik predykcji modelu oraz jego jakości, gdyż, będzie on pracował, na dancyh z dużo mniejszą różnicą rozkładów cech pomiędzy zbiorem treningowym, a testowym, przez to predyckje na zbiorze testowym będą znacznie lepsze, i trafniejsze.